In [1]:
import torch
from tqdm import tqdm
import pandas as pd
import open_clip
from bias_explorer.utils import dataloader, system
from bias_explorer.operations import analyze

In [2]:
conf = dataloader.load_json("./conf.json")

In [3]:
df_dict = analyze.collect_analysis(conf)

In [4]:
model = conf['Model']
backbone = "ViT-L-14"
metric = conf['Metric']
report_path = conf['Reports']
ds_path = f"{report_path}/{model}/{backbone}"
label = system.grab_label_name(conf['Labels'])
ln = f"{conf['Target']}_{label}"
agg_reports = analyze.collect_reports(ds_path, ln, metric, True)

In [12]:
agg_reports['openai']

,Mode,accuracy,0-2,3-9,10-19,20-29,30-39,40-49,50-59,60-69,more than 70,Male,Female,East Asian,White,Latino_Hispanic,Southeast Asian,Black,Indian,Middle Eastern
0,openAI Aggregation,0.6689,0.593,0.66,0.7197,0.6564,0.6725,0.6851,0.6319,0.676,0.7119,0.6614,0.6773,0.7019,0.482,0.6691,0.7046,0.8535,0.7361,0.5848


In [11]:
df_dict['ViT-L-14']

,datasource,Mode,accuracy,0-2,3-9,10-19,20-29,30-39,40-49,50-59,...,Male,Female,East Asian,White,Latino_Hispanic,Southeast Asian,Black,Indian,Middle Eastern,Race Gap
0,openai,Top 04,0.6732,0.5829,0.6578,0.7214,0.6621,0.6803,0.6859,0.6420,...,0.6742,0.6720,0.6910,0.4935,0.6303,0.7138,0.8586,0.7414,0.6460,0.1886
1,laion400m_e32,Avg Sum,0.6665,0.6382,0.6460,0.7180,0.6642,0.6730,0.6800,0.6181,...,0.6614,0.6722,0.6910,0.7050,0.5059,0.5781,0.8985,0.7058,0.5401,0.1547
2,datacomp_xl_s13b_b90k,Top 06,0.6204,0.6332,0.6121,0.6257,0.6221,0.6343,0.6061,0.6080,...,0.6146,0.6269,0.7019,0.8456,0.1830,0.6000,0.8715,0.6649,0.3598,0.4208
3,laion2b_s32b_b82k,Avg Sum,0.6930,0.6080,0.6910,0.7231,0.6909,0.6974,0.7110,0.6457,...,0.6859,0.7009,0.7755,0.7041,0.5903,0.6431,0.8374,0.7289,0.5335,0.1540
4,commonpool_xl_s13b_b90k,Top 01,0.5881,0.5628,0.5922,0.5927,0.5721,0.6060,0.5950,0.5817,...,0.5853,0.5912,0.5819,0.7170,0.3401,0.6332,0.8265,0.5765,0.3615,0.2366


In [7]:
archs_and_dsources = dataloader.load_json("./archs_and_datasources.json")

In [10]:
backbones

['ViT-B-16', 'ViT-B-32', 'ViT-L-14', 'ViT-H-14', 'ViT-g-14']